<a href="https://colab.research.google.com/github/eftekhar-hossain/SKBI_Training/blob/main/Session_15_NLP_(word2vec).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> <u> <font color='red'> Training on AI and ML with Python (Intermediate) </font> </u> </h1> </center>

#Session-15: Word2Vec (Word Embeddings)


###Objective: 
1. *Develop a Word2Vec model on a text corpus.*
2. *Generate word vectors using word2vec algorithm.*

 



In [1]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import tensorflow as tf
from tensorflow import keras

CPU times: user 3.21 s, sys: 390 ms, total: 3.6 s
Wall time: 4.3 s


## Fetching the Dataset
Now we will get the dataset form our google drive. A class array will be created which contains the name of the classes we are interested to work with.

In [2]:
!wget -O news.csv https://www.dropbox.com/s/12yza2h4bja9fqy/News_12Class.csv?dl=0

--2023-06-02 15:10:37--  https://www.dropbox.com/s/12yza2h4bja9fqy/News_12Class.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/12yza2h4bja9fqy/News_12Class.csv [following]
--2023-06-02 15:10:37--  https://www.dropbox.com/s/raw/12yza2h4bja9fqy/News_12Class.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbe63491e1507241e8ed3bcbdd9.dl.dropboxusercontent.com/cd/0/inline/B9M_WySHbiKoo69CiT791_ezkC7mZL-hKik0TSrMCtisV5AiC2nQyr22Knfw8l_QdzT-962TjYU9ltXUfFGwyF5qVvS8v3ook6MYPn4JOHEBWmHLbJKnrU35HgoyMG3rEdWCaYidrGlgN-_p89C70yq1zs65L1pBHkKpYqov8rsxdg/file# [following]
--2023-06-02 15:10:37--  https://ucbe63491e1507241e8ed3bcbdd9.dl.dropboxusercontent.com/cd/0/inline/B9M_WySHbiKoo69CiT791_ezkC7mZL-hKik0TSrMCtisV5AiC2nQyr22Knfw

In [3]:
%%time
class_names = ['Accident', 'Art', 'Crime', 'Economics', 'Education', 'Entertainment',
               'Environment', 'International', 'Politics', 'Science_tech', 'Sports', 'Suspicious']
               
dataset = pd.read_csv("news.csv")
dataset

CPU times: user 5.17 s, sys: 428 ms, total: 5.6 s
Wall time: 5.69 s


,Unnamed: 0,Text,Category
0,0,নাটোরের বড়াইগ্রাম উপজেলায় মাটির দেয়াল চাপা পড়...,Accident
1,1,চট্টগ্রামের আনোয়ারায় পানিতে ডুবে এক শিশুর মৃত...,Accident
2,2,পূর্ব চীনের শাংডং প্রদেশে কয়লা খনিতে বন্যার প...,Accident
3,3,ঢাকার ধামরাই উপজেলায় ঢাকা-আরিচা মহাসড়কে বুধবা...,Accident
4,4,গত তিনমাসে এ সড়কে দুর্ঘটনায় ৪২ জনের মৃত্যু হয়ে...,Accident
...,...,...,...
95861,95861,খেলা শুরুর ৪০ সেকেন্ডের মধ্যে প্রথম গোল। ৮৯ ম...,Sports
95862,95862,শনিবার বিকাল ৫টায় ঢাকার কেন্দ্রীয় শহীদ মিনার চ...,Sports
95863,95863,নিউজিল্যান্ডের জয়রথ ছুটছেই। ম্যাককালাম-ঝড় উঠছে...,Sports
95864,95864,এফসি কোপেনহেগেনের সঙ্গে ফিরতি খেলায় গোলশূন্য ...,Sports


In [4]:
labels = np.array(dataset['Category'])
print('Total size of the dataset : ',len(dataset))

Total size of the dataset :  95866


The total size of the dataset is 90996. We will build our word embedding model on this dataset

## Data Preprocessing
Here we will process the data by removing punctuation and stop words from the dataset. It is really a important part for the successful implementation of our word embedding model.

### Removing stop words and punctuation
I do not think when we are implementing words embedding there are any need of removing stop words. This effect our cause of interest a little. So there is not need to remove this stop words for this case. But a efficient way of removing stop words are shown below.

In [5]:
%%time
total_words = set() ## list of all words in our dataset

def creating_total_words(words):
     for w in words:
          total_words.add(w)

processed_data = [] ##List of all the processed text

for i in range(len(dataset)):
    text = re.sub('[^\u0980-\u09FF]',' ', dataset['Text'][i]) ## removing punctions 
    ## removing stop words from the texts
    words =text.split()
    creating_total_words(words)
    processed_data.append(words)

CPU times: user 18.6 s, sys: 965 ms, total: 19.6 s
Wall time: 19.8 s


In [6]:
# Finding the size of the vocabulary
print("Total Number of Words: ",len(total_words))

Total Number of Words:  429370


Here exists a special trick that inceease the total runtime of the system. So this code is removed and executed separately

In [ ]:
##Look at the a fraction of the dataset after preprocessing

for i in range(10):
    print(dataset['Text'][i],'\n',processed_data[i],'\n\n')

 নাটোরের বড়াইগ্রাম উপজেলায় মাটির দেয়াল চাপা পড়ে মা ও ছেলের মৃত্যু হয়েছে। সোমবার রাত ৮টার দিকে উপজেলার মেরিগাছা বাজারের পাশে জেলে পাড়ায় এ দুর্ঘটনা ঘটে বলে প্রত্যক্ষদর্শীরা জানিয়েছে। নিহতরা হলেন- রাজিয়া বেগম (৫৫) ও তার ছেলে হাবিবুর রহমান হবি (৩৫)। রাজিয়া বেগমের স্বামী নূর মোহাম্মদ বিডিনিউজ টোয়েন্টিফোর ডটকমকে জানান, রাতে মা ও ছেলে বাড়ির উঠানে বসে মাছ ধরার চাঁই তৈরি করছিলেন। এ সময় ঘরের পুরোনো মাটির দেয়াল ধসে তাদের ওপর পড়ে। "হাসপাতালে নেওয়ার পথে রাত ৯টার দিকে রাজিয়া মারা যান। রাত পৌনে ১০টার দিকে মারা যায় হাবিব।"  
 ['নাটোরের', 'বড়াইগ্রাম', 'উপজেলায়', 'মাটির', 'দেয়াল', 'চাপা', 'পড়ে', 'মা', 'ও', 'ছেলের', 'মৃত্যু', 'হয়েছে', 'সোমবার', 'রাত', '৮টার', 'দিকে', 'উপজেলার', 'মেরিগাছা', 'বাজারের', 'পাশে', 'জেলে', 'পাড়ায়', 'এ', 'দুর্ঘটনা', 'ঘটে', 'বলে', 'প্রত্যক্ষদর্শীরা', 'জানিয়েছে', 'নিহতরা', 'হলেন', 'রাজিয়া', 'বেগম', '৫৫', 'ও', 'তার', 'ছেলে', 'হাবিবুর', 'রহমান', 'হবি', '৩৫', 'রাজিয়া', 'বেগমের', 'স্বামী', 'নূর', 'মোহাম্মদ', 'বিডিনিউজ', 'টোয়েন্টিফোর', 'ডটকমকে', 'জানান', 'রাতে', 'মা', 'ও', 'ছেলে', 'বাড়

# Creating Word Embedding Model


## Defining the Model
Here we will define the model and necessary hyperparameters. Necessity of this hyperparameters will be briefly describe in the paper.

In [7]:
%%time
from gensim.models import Word2Vec
#different hyperparameter that can be used to test the system 

word2vec = Word2Vec(processed_data, vector_size=10,  min_count=5, window=5, workers=40, seed=1, negative=5)

CPU times: user 4min 35s, sys: 1.58 s, total: 4min 36s
Wall time: 2min 41s


In [8]:
# Save model in txt format
word2vec.wv.save_word2vec_format('test_w2v.txt', binary=False)

In [9]:
# load the embedding vectors
from gensim.models import KeyedVectors

word_embedding_model = KeyedVectors.load_word2vec_format('test_w2v.txt', binary=False,encoding='utf-8')

In [11]:
vocabulary = word2vec.wv.index_to_key
print(len(vocabulary))

123771


## Creating necessary List and Arrays
Create the necessary lists which will be used for further processing with our model.

In [12]:
a = vocabulary[1]
a

'করে'

In [14]:
print(word2vec.wv[a])

[-2.520959  -3.1368358  8.518831  -3.8995786 -1.3587413 -1.0275813
 -4.1509795 -1.0123392  5.7859306  1.5043509]


In [15]:
%%time

word_to_vec_map = {}
words_to_index = {}
index_to_words = {}
frequency_of_words = {}

c =1
for w in vocabulary:
  # Word ----> vector
  word_to_vec_map[w] = word2vec.wv[w]
  words_to_index[w] = c
  index_to_words[c] = w
  c = c+1

print(len(vocabulary))

123771
CPU times: user 253 ms, sys: 15 ms, total: 268 ms
Wall time: 269 ms


## Creating List of Check Words
Here we will create a list  through which we evaluate our model.

In [16]:
# get 10 words
check_words = ["রহিম", "কুকুর", "চুয়েট", "ভাল", "ঢাকা", "রোনালদো", "মার্চ"]

# find the word vectors of those 10 words
for w in check_words:
    if w in vocabulary:
      print(w)
      print(word_to_vec_map[w])
      

রহিম
[-1.0247257  5.268694   2.9054224  0.7851943  0.657778   1.4365566
  3.2542353 -1.9803832 -0.7376157  1.3801023]
কুকুর
[-2.073287    0.993882   -0.61210626 -0.7711771  -1.0458134  -1.0096794
 -1.1125343  -0.9464869  -3.461303   -2.8526661 ]
চুয়েট
[ 1.2427043  -0.63197935  0.86258     1.1703308   1.3583671   0.5256293
  0.87376255  0.26976973 -1.1226315  -0.06667692]
ভাল
[-0.18039887 -4.3269296  -2.3897095   0.0533834  -5.8803177  -3.005363
 -0.23671378 -3.9924545   1.5548489  -2.5839732 ]
ঢাকা
[ 1.6186959  -1.0518805   3.542541    8.586368    3.3728044   3.050725
 -0.67931587 -4.736907    2.4829617   1.0646527 ]
রোনালদো
[-3.6945329   0.18026048 -2.2775502   0.08412847 -4.593818    1.2499377
  3.8549209  -2.2410965   2.964369   -2.5186524 ]
মার্চ
[-0.52860177 -1.3489392  -4.9863467   5.097474   -2.2498896   8.517255
  2.2014496   1.1256986   4.400125    7.8650084 ]


## Finding most frequent words and similar words

In [17]:
%%time

## Finding the similar words of this frequent words
words_to_print = []
for w in check_words:
  print('\n',w)
  words_to_print.append(w)
  similar_words = word2vec.wv.most_similar(w, topn=10)
  for wr, vec in similar_words:
    words_to_print.append(wr)
    print(wr, vec)


 রহিম
মিশুক 0.9662218689918518
আমিনের 0.9546286463737488
মহিউদ্দিনকে 0.9524199366569519
পিন্টু 0.9497205018997192
খালেদের 0.9458311200141907
সাঈদ 0.9430427551269531
হায়দারের 0.942508339881897
রহিমকে 0.9408461451530457
মুনীরের 0.9396954774856567
সিদ্দিকের 0.9371025562286377

 কুকুর
গায়ে 0.9775260090827942
মেয়েরা 0.9769939184188843
গালে 0.9750528335571289
পিঠে 0.9748523831367493
পায়ের 0.9747335910797119
ছোবল 0.9728796482086182
রুমাল 0.9701821804046631
পোষা 0.9689037203788757
কোমর 0.9666697382926941
নিঃশ্বাসের 0.966552734375

 চুয়েট
ব্রাক 0.9726178050041199
ভেটেরিনারি 0.9718738794326782
রুয়েট 0.9700735211372375
বুয়েট 0.969963014125824
বিএবি 0.9687003493309021
ইন্সটিটিউটের 0.9632660150527954
ইলেকট্রিক্যাল 0.9625769853591919
এজেন্টস 0.9623699188232422
ওনার্স 0.9622644186019897
শাবিপ্রবি 0.9599798917770386

 ভাল
ভালো 0.9948699474334717
ভালো 0.973506510257721
শুধু 0.9669355750083923
যেটা 0.9652225971221924
তাই 0.9568595886230469
খারাপ 0.9506151080131531
সেটাই 0.9484941959381104
বদলে 0.9